  In this phase integrated Car Kivy environment with T3D learning and used CNN

  ### **Sensory data** : Taken 3 patches in front of the car

  >- left 30 degrees ; 40x40 
  >- straight 0 degrees ; 40x40
  >- right 30 degrees; 40x40

  > then merged them to create 3 channel input to the CNN Network


  ### **Parameters used**
  >- **Action Dimension** : 1 - angle of rotation
  >- **State Dimension** : 1200
  >>- 3 channels of 40x40 = 1200 fed to the CNN

  >- **Rewards**
  >>- On sand = - 1
  >>- Car hits the edges = - 1
  >>- If on Road and distance is reducing = + 1
  >>- On Road = + 0.8

  ### **Network**

  Convblk1 -> MaxPool -> ConvBlk -> GAP -> FC 

  ### **Observations**

  Car started to rotate after few episodes.Below is the diagnostics and steps taken. This could mean that only extreme angle (-max_action or + max_action) was predicted. This meant network was unstable. I could not be sure if this was for exploding gradients or vanishing gradients. Took the below approach



  >- Enabled training logs and looked at buffer values ; predicted rotations
  >- Compared training logs between run using T3D + FC + Car(successful) vs T3D + CNN + Car(un-successful)

            Target Q's should be negative ( they are low positive in CNN network)
            Current Q's should be negative ( they are low positive in CNN network)
            Critic loss should be low positive (they are high positive in CNN network)
            Action Loss should be large positive ( they are los positive in CNN network)
            Q1 should be large negative ( they are small negative in CNN network)

  >- Led me to realize i had not used Batch Normalization across layers. Enabled that and network improved, but was not staying on the road



## Potential Next Steps

>- Reduce the number of layers in CNN - to see if vanishing gradient in an issue
>- Use only one channel 
>- Get the orientation from destination and feed that to the network and concatenate after feature extraction from CNN


In [0]:
# Self Driving Car


# Replacing Signal with CNN


'''

Issues faced and approaches taken

1. Car Rotation :
      Potential approaches - try playing with data in the buffer - it uses that to learn
      Tweat rewards
      Tweat Done 
      Allow it to explore longer - to load buffer with more data to learn from 
    
      
      Approach : Tweat rewards
          Rewards :Logged the rewards when moving on road vs sand and tweaked it
          Changed the max_action to 10 

      Observation :

          Continued rotating - faster

      Approach : Tweak done 
          - if on sand done = False
          - If on sand and old state and new state are on sand then done
          - Change velocity to 2 - to move it out

      Observation :
            stood in same place ; did not move
            Soemthing changed - improved slightly
            Came back to rotating - once network action kicks in

       Approach : Change signal actoivations to sigmoid - not sure if there is gradient explosion

       Observation :
            No change

    Diagonistics
          Print avergage of done and reward from buffer
          Print avergage of action during training

     Observations
          actor not getting loss - so not changing angle

          Target Q's should be negative ( they are low positive - why ?)
          Current Q's should be negative ( they are low positive - why ?)
          Critic loss should be low positive (they are high positive - why ?)
          Action Loss should be large positive ( they are los positive - why ?)
       




'''





# Importing the libraries
import numpy as np
import random as R
from random import random, randint
import matplotlib.pyplot as plt
import time

# Importing the Kivy packages
from kivy.app import App
from kivy.uix.widget import Widget
from kivy.uix.button import Button
from kivy.graphics import Color, Ellipse, Line
from kivy.config import Config
from kivy.properties import NumericProperty, ReferenceListProperty, ObjectProperty
from kivy.vector import Vector
from kivy.clock import Clock
from kivy.core.image import Image as CoreImage
from PIL import Image as PILImage
from kivy.graphics.texture import Texture
# ASV Starts
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from collections import deque
#from t3d import TD3

# ASV Ends

# Importing the Dqn object from our AI in ai.py
from ai import Dqn

# Adding this line if we don't want the right click to put a red point
Config.set('input', 'mouse', 'mouse,multitouch_on_demand')
Config.set('graphics', 'resizable', False)
Config.set('graphics', 'width', '1429')
Config.set('graphics', 'height', '660')

# Introducing last_x and last_y, used to keep the last point in memory when we draw the sand on the map
last_x = 0
last_y = 0
n_points = 0
length = 0

# Getting our AI, which we call "brain", and that contains our neural network that represents our Q-function
brain = Dqn(5,3,0.9)
action2rotation = [0,5,-5]
last_reward = 0
scores = []
im = CoreImage("./images/MASK1.png")

# textureMask = CoreImage(source="./kivytest/simplemask1.png")


#Start ASV




# Initialize the Experience Replay memory
class ReplayBuffer(object):

  def __init__(self, max_size=1e6):
    self.storage = []
    self.max_size = max_size
    self.ptr = 0

  def add(self, transition):
    if len(self.storage) == self.max_size:
      self.storage[int(self.ptr)] = transition
      self.ptr = (self.ptr + 1) % self.max_size
    else:
      self.storage.append(transition)
    #print('length of storage : {}'.format(len(self.storage)))

  def sample(self, batch_size):
    ind = np.random.randint(0, len(self.storage), size=batch_size)
    batch_states, batch_next_states, batch_actions, batch_rewards, batch_dones = [], [], [], [], []
    for i in ind: 
      state, next_state, action, reward, done = self.storage[i]
      batch_states.append(np.array(state, copy=False))
      batch_next_states.append(np.array(next_state, copy=False))
      batch_actions.append(np.array(action, copy=False))
      batch_rewards.append(np.array(reward, copy=False))
      batch_dones.append(np.array(done, copy=False))
    return np.array(batch_states), np.array(batch_next_states), np.array(batch_actions), np.array(batch_rewards).reshape(-1, 1), np.array(batch_dones).reshape(-1, 1)




class Actor_cnn_v2(nn.Module):
  
  def __init__(self, state_dim, action_dim, max_action):
    super(Actor_cnn_v2, self).__init__()
   

    # CONVOLUTION BLOCK 1
    self.convblock1 = nn.Sequential(
        nn.Conv2d(in_channels=3, out_channels=8, kernel_size=(3, 3), padding=0, bias=False),
        nn.BatchNorm2d(8),
        nn.ReLU()
        #nn.ReLU()
    ) # output_size = 18
    self.convblock2 = nn.Sequential(
        nn.Conv2d(in_channels=8, out_channels=16, kernel_size=(3, 3), padding=0, bias=False),
        nn.BatchNorm2d(16),
        nn.Dropout2d(p=0.1),
        nn.ReLU()
        #nn.ReLU()
    ) # output_size = 16

    # TRANSITION BLOCK 1
    self.pool1 = nn.MaxPool2d(2, 2) # output_size = 8


    # CONVOLUTION BLOCK 2
    self.convblock3 = nn.Sequential(
        nn.Conv2d(in_channels=16, out_channels=8, kernel_size=(3, 3), padding=0, bias=False),
        nn.BatchNorm2d(8),
        nn.ReLU()
        #nn.ReLU()
    ) # output_size = 6
    self.convblock4 = nn.Sequential(
        nn.Conv2d(in_channels=8, out_channels=4, kernel_size=(3, 3), padding=0, bias=False),
        nn.BatchNorm2d(4),
        nn.Dropout2d(p=0.1),
        nn.ReLU()
        #nn.ReLU()
    ) # output_size = 4

   
    self.avgpool = nn.AvgPool2d(4)

    self.linblock = nn.Sequential(
        nn.Linear(4,400),
        nn.ReLU(),
        nn.Linear(400,300),
        nn.ReLU(),
        nn.Linear(300,action_dim)
    ) 



       
   
    self.max_action = max_action
  
  def forward(self, x):

    x = np.reshape(x,(-1,3,20,20))

    #print('mean and std'.format(x.mean(),x.std()))
    #transforms.Normalize(x,(0.5))
    
    x = self.convblock1(x)
    x = self.convblock2(x)
    x = self.pool1(x)
    x = self.convblock3(x)
    x = self.convblock4(x)
    x = self.avgpool(x)
    x = x.view(-1, 4)
    x = self.max_action * torch.tanh(self.linblock(x))
    
    return x
  



class Critic_cnn_v2(nn.Module):

  def __init__(self, state_dim, action_dim):
    super(Critic_cnn_v2, self).__init__()
   

    # CONVOLUTION BLOCK 1
    self.convblock1 = nn.Sequential(
        nn.Conv2d(in_channels=3, out_channels=8, kernel_size=(3, 3), padding=0, bias=False),
        nn.BatchNorm2d(8),
        nn.ReLU()
        #nn.ReLU()
    ) # output_size = 18
    self.convblock2 = nn.Sequential(
        nn.Conv2d(in_channels=8, out_channels=16, kernel_size=(3, 3), padding=0, bias=False),
        nn.BatchNorm2d(16),
        nn.Dropout2d(p=0.1),
        nn.ReLU()
        #nn.ReLU()
    ) # output_size = 16

    # TRANSITION BLOCK 1
    self.pool1 = nn.MaxPool2d(2, 2) # output_size = 8


    # CONVOLUTION BLOCK 2
    self.convblock3 = nn.Sequential(
        nn.Conv2d(in_channels=16, out_channels=8, kernel_size=(3, 3), padding=0, bias=False),
        nn.BatchNorm2d(8),
        nn.ReLU()
        #nn.ReLU()
    ) # output_size = 6
    self.convblock4 = nn.Sequential(
        nn.Conv2d(in_channels=8, out_channels=4, kernel_size=(3, 3), padding=0, bias=False),
        nn.BatchNorm2d(4),
        nn.Dropout2d(p=0.1),
        nn.ReLU()
        #nn.ReLU()
    ) # output_size = 4

   
    self.avgpool = nn.AvgPool2d(4)

    self.linblock = nn.Sequential(
        nn.Linear(5,400),
        nn.ReLU(),
        nn.Linear(400,300),
        nn.ReLU(),
        nn.Linear(300,action_dim)
      
    ) 





    # CONVOLUTION BLOCK 1
    self.convblock1a = nn.Sequential(
        nn.Conv2d(in_channels=3, out_channels=8, kernel_size=(3, 3), padding=0, bias=False),
        nn.BatchNorm2d(8),
        nn.ReLU()
        #nn.ReLU()
    ) # output_size = 18
    self.convblock2a = nn.Sequential(
        nn.Conv2d(in_channels=8, out_channels=16, kernel_size=(3, 3), padding=0, bias=False),
        nn.BatchNorm2d(16),
        nn.Dropout2d(p=0.1),
        nn.ReLU()
        #nn.ReLU()
    ) # output_size = 16

    # TRANSITION BLOCK 1
    self.pool1a = nn.MaxPool2d(2, 2) # output_size = 8


    # CONVOLUTION BLOCK 2
    self.convblock3a = nn.Sequential(
        nn.Conv2d(in_channels=16, out_channels=8, kernel_size=(3, 3), padding=0, bias=False),
        nn.BatchNorm2d(8),
        nn.ReLU()
        #nn.ReLU()
    ) # output_size = 6
    self.convblock4a = nn.Sequential(
        nn.Conv2d(in_channels=8, out_channels=4, kernel_size=(3, 3), padding=0, bias=False),
        nn.BatchNorm2d(4),
        nn.Dropout2d(p=0.1),
        nn.ReLU()
        #nn.ReLU()
    ) # output_size = 4

   
    self.avgpoola = nn.AvgPool2d(4)

    self.linblocka = nn.Sequential(
        nn.Linear(5,400),
        nn.ReLU(),
        nn.Linear(400,300),
        nn.ReLU(),
        nn.Linear(300,action_dim)
      
    ) 


  
  def forward(self, x, u):
   
    x0 = np.reshape(x,(-1,3,20,20))

    x1 = self.convblock1(x0)
    x1 = self.convblock2(x1)
    x1 = self.pool1(x1)
    x1 = self.convblock3(x1)
    x1 = self.convblock4(x1)
    x1 = self.avgpool(x1)
    x1 = x1.view(-1, 4)
    x1u1 = torch.cat([x1, u], 1)
    x1 = self.linblock(x1u1)
    

    x2 = self.convblock1a(x0)
    x2 = self.convblock2a(x2)
    x2 = self.pool1a(x2)
    x2 = self.convblock3a(x2)
    x2 = self.convblock4a(x2)
    x2 = self.avgpoola(x2)
    x2 = x2.view(-1, 4)
    x2u1 = torch.cat([x2, u], 1)
    x2 = self.linblocka(x2u1)


    return x1, x2

  def Q1(self, x, u):

    x0 = np.reshape(x,(-1,3,20,20))

    x1 = self.convblock1(x0)
    x1 = self.convblock2(x1)
    x1 = self.pool1(x1)
    x1 = self.convblock3(x1)
    x1 = self.convblock4(x1)
    x1 = self.avgpool(x1)
    x1 = x1.view(-1, 4)
    x1u1 = torch.cat([x1, u], 1)
    x1 = self.linblock(x1u1)
    

    return x1





# Training

class TD3(object):

  #device = 'cpu'
  
  def __init__(self, state_dim, action_dim, max_action):
    self.actor = Actor_cnn_v2(state_dim, action_dim, max_action).to(device)
    self.actor_target = Actor_cnn_v2(state_dim, action_dim, max_action).to(device)
    self.actor_target.load_state_dict(self.actor.state_dict())
    self.actor_optimizer = torch.optim.Adam(self.actor.parameters())
    self.critic = Critic_cnn_v2(state_dim, action_dim).to(device)
    self.critic_target = Critic_cnn_v2(state_dim, action_dim).to(device)
    self.critic_target.load_state_dict(self.critic.state_dict())
    self.critic_optimizer = torch.optim.Adam(self.critic.parameters())
    self.max_action = max_action

  def select_action(self, state):
    state = torch.Tensor(state.reshape(1, -1)).to(device)
    return self.actor(state).cpu().data.numpy().flatten()

  def train(self, replay_buffer, iterations, batch_size=100, discount=0.99, tau=0.005, policy_noise=0.2, noise_clip=0.5, policy_freq=2):
    
    for it in range(iterations):
      
      # Step 4: We sample a batch of transitions (s, s’, a, r) from the memory
      batch_states, batch_next_states, batch_actions, batch_rewards, batch_dones = replay_buffer.sample(batch_size)
      #print(round(np.mean(batch_actions),2),np.mean(batch_rewards), np.mean(batch_dones))
      state = torch.Tensor(batch_states).to(device)
      next_state = torch.Tensor(batch_next_states).to(device)
      action = torch.Tensor(batch_actions).to(device)
      reward = torch.Tensor(batch_rewards).to(device)
      done = torch.Tensor(batch_dones).to(device)
      
      # Step 5: From the next state s’, the Actor target plays the next action a’
      #print(next_state.shape)
      #start = time.time()
      next_action = self.actor_target(next_state)
      #print(type(next_action),next_action.shape)
      #print('predicted actions mean : {}'.format(torch.mean(next_action),2))
      #at_fwd = time.time() - start
      #print(next_action.shape)

      # Step 6: We add Gaussian noise to this next action a’ and we clamp it in a range of values supported by the environment
      noise = torch.Tensor(batch_actions).data.normal_(0, policy_noise).to(device)
      noise = noise.clamp(-noise_clip, noise_clip)
      #print('noise shape: {}'.format(noise.shape))
      next_action = (next_action + noise).clamp(-self.max_action, self.max_action)
      
      
      # Step 7: The two Critic targets take each the couple (s’, a’) as input and return two Q-values Qt1(s’,a’) and Qt2(s’,a’) as outputs
      #start = time.time()
      #print('snew shape {} next action shape : {}'.format(next_state.shape,next_action.shape))
      target_Q1, target_Q2 = self.critic_target(next_state, next_action)
      #print('Target Qs : {} {}'.format(torch.mean(target_Q1),torch.mean(target_Q2)))
      #ct_fwd = time.time() - start
      # Step 8: We keep the minimum of these two Q-values: min(Qt1, Qt2)
      target_Q = torch.min(target_Q1, target_Q2)
      
      # Step 9: We get the final target of the two Critic models, which is: Qt = r + γ * min(Qt1, Qt2), where γ is the discount factor
      target_Q = reward + ((1 - done) * discount * target_Q).detach()
      
      #print('state shape: {} action shape : {}'.format(state.shape,action.shape))
      # Step 10: The two Critic models take each the couple (s, a) as input and return two Q-values Q1(s,a) and Q2(s,a) as outputs
      #start = time.time()
      current_Q1, current_Q2 = self.critic(state, action)
      #print('Current Qs : {} {}'.format(torch.mean(current_Q1),torch.mean(current_Q2)))
      #cm_fwd = time.time() - start
      
      # Step 11: We compute the loss coming from the two Critic models: Critic Loss = MSE_Loss(Q1(s,a), Qt) + MSE_Loss(Q2(s,a), Qt)
      critic_loss = F.mse_loss(current_Q1, target_Q) + F.mse_loss(current_Q2, target_Q)
      #print('critic_loss : {}'.format(critic_loss))
      # Step 12: We backpropagate this Critic loss and update the parameters of the two Critic models with a SGD optimizer
      #start = time.time()
      self.critic_optimizer.zero_grad()
      critic_loss.backward()
      self.critic_optimizer.step()
      #c_bp = time.time() - start

      # Step 13: Once every two iterations, we update our Actor model by performing gradient ascent on the output of the first Critic model
      if it % policy_freq == 0:
        #start = time.time()
        actor_loss = -self.critic.Q1(state, self.actor(state)).mean()
        #print('actor_loss : {}'.format(actor_loss))
        self.actor_optimizer.zero_grad()
        actor_loss.backward()
        self.actor_optimizer.step()
        
        # Step 14: Still once every two iterations, we update the weights of the Actor target by polyak averaging
        for param, target_param in zip(self.actor.parameters(), self.actor_target.parameters()):
          target_param.data.copy_(tau * param.data + (1 - tau) * target_param.data)
        
        # Step 15: Still once every two iterations, we update the weights of the Critic target by polyak averaging
        for param, target_param in zip(self.critic.parameters(), self.critic_target.parameters()):
          target_param.data.copy_(tau * param.data + (1 - tau) * target_param.data)
        #pol_freq = time.time() - start
      #print(' action target : {} critic target {} critic model {} critic backprop : {} policy freq {}'.format(at_fwd,ct_fwd,cm_fwd,c_bp,pol_freq))
      #print(' Iterations during training : {}'.format(it))
      #logging
      if it % 25 == 0:
          #print(round(np.mean(batch_actions),2),np.mean(batch_rewards), np.mean(batch_dones))
          print(batch_states.shape,batch_next_states.shape,batch_actions.shape,batch_rewards.shape, batch_dones.shape)
          print('Mean batch obs :{}'.format(round(np.mean(batch_states),2)))
          print('Mean batch new obs  :{}'.format(round(np.mean(batch_next_states),2)))
          print('Mean batch actions :{}'.format(round(np.mean(batch_actions),2)))
          print('Mean batch rewards :{}'.format(round(np.mean(batch_rewards),2)))
          print('Mean batch dones :{}'.format(round(np.mean(batch_rewards),2)))

          print('Predicted Rotations/ Next Action : {}'.format(torch.mean(next_action)))

          print('Target Qs  : {} {}'.format(torch.mean(target_Q1),torch.mean(target_Q2)))
          print(' Critic Target Q : {}'.format(torch.mean(target_Q)))
          print('Current Qs : {} {}'.format(torch.mean(current_Q1),torch.mean(current_Q2)))
          print('critic_loss : {}'.format(critic_loss))

          print('Current state : {}'.format(torch.mean(state)))
          print('Current Rotations/action : {}'.format(torch.mean(action)))
          print('Mean Critic Model Q1/ Action Loss : {}'.format(actor_loss))

      
        
          
          
          





  # Making a save method to save a trained model
  def save(self, filename, directory):
    torch.save(self.actor.state_dict(), '%s/%s_actor.pth' % (directory, filename))
    torch.save(self.critic.state_dict(), '%s/%s_critic.pth' % (directory, filename))
  
  # Making a load method to load a pre-trained model
  def load(self, filename, directory):
    self.actor.load_state_dict(torch.load('%s/%s_actor.pth' % (directory, filename)))
    self.critic.load_state_dict(torch.load('%s/%s_critic.pth' % (directory, filename)))



 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

env_name = "Car_on_Road" 
seed = 0 # Random seed number
start_timesteps = 1e2 # Number of iterations/timesteps before which the model randomly chooses an action, and after which it starts to use the policy network
eval_freq = 5e3 # How often the evaluation step is performed (after how many timesteps)
max_timesteps = 5e5 # Total number of iterations/timesteps
save_models = True # Boolean checker whether or not to save the pre-trained model
expl_noise = 0.1 # Exploration noise - STD value of exploration Gaussian noise
batch_size = 100 # Size of the batch
discount = 0.99 # Discount factor gamma, used in the calculation of the total discounted reward
tau = 0.005 # Target network update rate
policy_noise = 0.2 # STD of Gaussian noise added to the actions for the exploration purposes
noise_clip = 0.5 # Maximum value of the Gaussian noise added to the actions (policy)
policy_freq = 2 # Number of iterations to wait before the policy network (Actor model) is updated


file_name = "%s_%s_%s" % ("TD3", env_name, str(seed))

if not os.path.exists("./results"):
  os.makedirs("./results")
if save_models and not os.path.exists("./pytorch_models"):
  os.makedirs("./pytorch_models")


torch.manual_seed(seed)
np.random.seed(seed)
state_dim = 1200 # 40*40
action_dim = 1
max_action = 10

# Building the brain
policy = TD3(state_dim, action_dim, max_action)
replay_buffer = ReplayBuffer()

global total_timesteps
global timesteps_since_eval
global episode_num
global done
global t0
global max_episode_steps

total_timesteps = 0
timesteps_since_eval = 0
episode_num = 0
done = True
t0 = time.time()
max_episode_steps = 1200


def env_action_space_sample():
    return round(R.uniform(-max_action,max_action), 3)


def get_action_list(log_prob):
    action_prob = torch.exp(log_prob)
    action_list = torch.multinomial(action_prob.squeeze(), 1).view(-1).numpy().tolist()
    return action_list
# ASV Ends


# Initializing the map
first_update = True
def init():
    global sand
    global goal_x
    global goal_y
    global first_update
    sand = np.zeros((longueur,largeur))
    img = PILImage.open("./images/mask.png").convert('L')
    sand = np.asarray(img)/255
    goal_x = 1420
    goal_y = 622
    first_update = False
    global swap
    swap = 0


# Initializing the last distance
last_distance = 0

# Creating the car class

class Car(Widget):
    
    angle = NumericProperty(0)
    rotation = NumericProperty(0)
    velocity_x = NumericProperty(0)
    velocity_y = NumericProperty(0)
    velocity = ReferenceListProperty(velocity_x, velocity_y)
    sensor1_x = NumericProperty(0)
    sensor1_y = NumericProperty(0)
    sensor1 = ReferenceListProperty(sensor1_x, sensor1_y)
    sensor2_x = NumericProperty(0)
    sensor2_y = NumericProperty(0)
    sensor2 = ReferenceListProperty(sensor2_x, sensor2_y)
    sensor3_x = NumericProperty(0)
    sensor3_y = NumericProperty(0)
    sensor3 = ReferenceListProperty(sensor3_x, sensor3_y)
    signal1 = NumericProperty(0)
    signal2 = NumericProperty(0)
    signal3 = NumericProperty(0)

    def move(self, rotation):
        self.pos = Vector(*self.velocity) + self.pos
        self.rotation = rotation
        self.angle = self.angle + self.rotation
        self.sensor1 = Vector(30, 0).rotate(self.angle) + self.pos
        self.sensor2 = Vector(30, 0).rotate((self.angle+30)%360) + self.pos
        self.sensor3 = Vector(30, 0).rotate((self.angle-30)%360) + self.pos
        self.signal1 = int(np.sum(sand[int(self.sensor1_x)-10:int(self.sensor1_x)+10, int(self.sensor1_y)-10:int(self.sensor1_y)+10]))/400.
        self.signal2 = int(np.sum(sand[int(self.sensor2_x)-10:int(self.sensor2_x)+10, int(self.sensor2_y)-10:int(self.sensor2_y)+10]))/400.
        self.signal3 = int(np.sum(sand[int(self.sensor3_x)-10:int(self.sensor3_x)+10, int(self.sensor3_y)-10:int(self.sensor3_y)+10]))/400.
        if self.sensor1_x>longueur-10 or self.sensor1_x<10 or self.sensor1_y>largeur-10 or self.sensor1_y<10:
            self.signal1 = 10.
        if self.sensor2_x>longueur-10 or self.sensor2_x<10 or self.sensor2_y>largeur-10 or self.sensor2_y<10:
            self.signal2 = 10.
        if self.sensor3_x>longueur-10 or self.sensor3_x<10 or self.sensor3_y>largeur-10 or self.sensor3_y<10:
            self.signal3 = 10.
       
    def eye(self, img):
        
        front_pos = Vector(30, 0).rotate(self.angle) + self.pos
        right_pos  = Vector(30, 0).rotate((self.angle+30)%360) + self.pos
        left_pos  = Vector(30, 0).rotate((self.angle-30)%360) + self.pos
        if car_at_edge(self.x,self.y,self.width,self.height) :
            state = list(np.ones((3,20,20)).ravel())
        else: 
            front_shot = img[int(front_pos.x)-10:int(front_pos.x)+10, int(front_pos.y)-10:int(front_pos.y)+10]
            right_shot = img[int(right_pos.x)-10:int(right_pos.x)+10, int(right_pos.y)-10:int(right_pos.y)+10]
            left_shot  = img[int(left_pos.x)-10:int(left_pos.x)+10, int(left_pos.y)-10:int(left_pos.y)+10]
            try:
              state = list(np.dstack((front_shot,right_shot,left_shot)).ravel())
            except ValueError:
              state = list(np.ones((3,20,20)).ravel())

        #Padding when getting closer to the edges
        if len(state) < state_dim :
            state = list(np.ones((3,20,20)).ravel())

        return state





class Ball1(Widget):
    pass
class Ball2(Widget):
    pass
class Ball3(Widget):
    pass

# Creating the game class

def car_at_edge(car_x,car_y,width,height):
    if ( (car_x< 70) or (car_x < width - 70) ) and ( (car_y < 70) or (car_y < height - 70) ) :
        return True
    else :
        return False 


class Game(Widget):

    car = ObjectProperty(None)
    ball1 = ObjectProperty(None)
    ball2 = ObjectProperty(None)
    ball3 = ObjectProperty(None)

    def serve_car(self):
        self.car.center = Vector(757,349) #self.center
        self.car.velocity = Vector(6, 0)





    def update(self, dt):

        global brain
        global last_reward
        global scores
        global last_distance
        global goal_x
        global goal_y
        global longueur
        global largeur
        global swap
        global sand
        global total_timesteps
        global timesteps_since_eval
        global episode_num
        global done
        global t0
        global max_episode_steps
        global episode_timesteps
        global episode_reward
        global new_signal

        longueur = self.width
        largeur = self.height
        if first_update:
            init()
            self.car.move(0)


        # ASV starts
        # If the episode is done
        if done:

            
        # If we are not at the very beginning, we start the training process of the model
            if total_timesteps != 0:

                print("Total Timesteps: {} Episode Num: {} Reward: {}".format(total_timesteps, episode_num, episode_reward))
                policy.train(replay_buffer, episode_timesteps, batch_size, discount, tau, policy_noise, noise_clip, policy_freq)


        
            
            # When the training step is done, we reset the state of the environment
           
            init()
            #Align car to center
            self.car.center = Vector(757,349) #self.center
            self.car.velocity = Vector(6, 0)
            xx = goal_x - self.car.x
            yy = goal_y - self.car.y
            orientation = Vector(*self.car.velocity).angle((xx,yy))/180.
            #last_signal = [self.car.signal1, self.car.signal2, self.car.signal3, orientation, -orientation]
            #print('After done')
            #print(int(self.car.sensor1),int(self.car.sensor1_y),int(self.car.sensor2_x),int(self.car.sensor2_y),int(self.car.sensor3_x),int(self.car.sensor3_y))
            #print(self.car.pos,self.car.sensor1,self.car.sensor2,self.car.sensor3)

    

            
            #old_obs = obs
            obs = self.car.eye(sand)

            
          
            
            # Set the Done to False
            done = False
            
            # Set rewards and episode timesteps to zero
            episode_reward = 0
            episode_timesteps = 0
            episode_num += 1
          
        # Before 10000 timesteps, we play random actions
        if total_timesteps < start_timesteps:
            rotation = env_action_space_sample()
            action = [rotation] #torch.from_numpy(np.reshape(np.array(rotation),(1,1))) [rotation]
            #print(' Rotated by using Random : {}'.format(action))

        else: # After 10000 timesteps, we switch to the model
            
            #old_obs = obs
            obs = self.car.eye(sand)
            
            


            #print(obs.shape)
            #print(int(self.car.sensor1_x),int(self.car.sensor1_y),int(self.car.sensor2_x),int(self.car.sensor2_y),int(self.car.sensor3_x),int(self.car.sensor3_y))
            action = policy.select_action(np.array(obs))
            #print(action)
            rotation = action.item()
            #rotation = action.item() # assuming one dimension
            #print('select_action : {}'.format(action))
            #rotation = action2rotation[np.argmax(action)]

            #print(action)
            #print(action,action.shape)
            # If the explore_noise parameter is not 0, we add noise to the action and we clip it
            if expl_noise != 0:
                action = (action + np.random.normal(0, expl_noise, 1)).clip(-max_action, max_action) # Clipping the rotation to 5 degrees
                #print(action.shape,action)
            print(' Rotated by using T3D : {}'.format(action))

        # The agent performs the action in the environment, then reaches the next state and receives the reward
        xx = goal_x - self.car.x
        yy = goal_y - self.car.y
        orientation = Vector(*self.car.velocity).angle((xx,yy))/180.
        #last_signal = [self.car.signal1, self.car.signal2, self.car.signal3, orientation, -orientation]

        obs = self.car.eye(sand)

        #print(last_signal)
        #action = brain.update(last_reward, last_signal)
        #scores.append(brain.score())

        
        #print(' Rotated by : {}'.format(rotation))
        self.car.move(rotation)
        #print('After move')
        #print(int(self.car.sensor1_x),int(self.car.sensor1_y),int(self.car.sensor2_x),int(self.car.sensor2_y),int(self.car.sensor3_x),int(self.car.sensor3_y))
        #print(int(self.car.pos),int(self.car.sensor1),int(self.car.sensor2),int(self.car.sensor3))
        #print(self.car.pos,self.car.sensor1,self.car.sensor2,self.car.sensor3)
        #new_signal = [self.car.signal1, self.car.signal2, self.car.signal3, orientation, -orientation]
        # Get right cuts
     
        new_obs = self.car.eye(sand)
        

  
        distance = np.sqrt((self.car.x - goal_x)**2 + (self.car.y - goal_y)**2)
        self.ball1.pos = self.car.sensor1
        self.ball2.pos = self.car.sensor2
        self.ball3.pos = self.car.sensor3

        if sand[int(self.car.x),int(self.car.y)] > 0:
            self.car.velocity = Vector(0.5, 0).rotate(self.car.angle)
            #print(1, goal_x, goal_y, distance, int(self.car.x),int(self.car.y), im.read_pixel(int(self.car.x),int(self.car.y)))

            # Tweaking Rewards #
            '''
            if (sum(new_obs) < state_dim) and (sum(obs) < state_dim) and (sum(new_obs) < sum(obs)):
                last_reward = 1
            else :
                last_reward = -1
            '''
            '''
            if (sum(new_obs) > 1000 ) and (sum(obs) > 1000) :
            #if sum(new_obs) == state_dim :
                done = True
            '''
            #done = True
            last_reward = -1
            #done = True

        else: # otherwise
            self.car.velocity = Vector(2, 0).rotate(self.car.angle)
            last_reward = 0.8 #0.8 # 0.8 #-0.2
            #print(0, goal_x, goal_y, distance, int(self.car.x),int(self.car.y), im.read_pixel(int(self.car.x),int(self.car.y)))
            if distance < last_distance:
                last_reward = 1 #1.0 #0.1
            # else:
            #     last_reward = last_reward +(-0.2)
            print('On Road')
            #print(int(self.car.sensor1_x),int(self.car.sensor1_y),int(self.car.sensor2_x),int(self.car.sensor2_y),int(self.car.sensor3_x),int(self.car.sensor3_y))
            #print(int(self.car.pos),int(self.car.sensor1),int(self.car.sensor2),int(self.car.sensor3))
            #print(np.sum(front_shot),np.sum(right_shot),np.sum(left_shot))
            print(int(self.car.x),int(self.car.y))
            print('new obs sum : {}'.format(sum(new_obs)))
            print('old obs sum : {}'.format(sum(obs)))


        #new_signal = [self.car.signal1, self.car.signal2, self.car.signal3, orientation, -orientation]

        if self.car.x < 5:
            self.car.x = 5
            last_reward = -1
            done = True # asv
        if self.car.x > self.width - 5:
            self.car.x = self.width - 5
            last_reward = -1
            done = True # asv
        if self.car.y < 5:
            self.car.y = 5
            last_reward = -1
            done = True # asv
        if self.car.y > self.height - 5:
            self.car.y = self.height - 5
            last_reward = -1
            done = True # asv

        if distance < 25:
            if swap == 1:
                goal_x = 1420
                goal_y = 622
                swap = 0
                done = True # asv
            else:
                goal_x = 9
                goal_y = 85
                swap = 1
                #done = True # asv
        last_distance = distance

        #new_obs, reward, done, _ = env.step(action)
        
        new_obs = new_obs
        reward  = last_reward


          # We check if the episode is done
        done_bool = 0 if episode_timesteps + 1 == max_episode_steps else float(done)
          
          # We increase the total reward
        episode_reward += reward
          
          # We store the new transition into the Experience Replay memory (ReplayBuffer)
        #print('obs : {} New Obs {} Action {} Reward {} Done {}'.format(len(obs),len(new_obs),type(action),type(reward),type(done_bool)))
        print('Before add to buffer new obs sum : {}'.format(sum(new_obs)))
        print('Before add to buffer obs sum : {}'.format(sum(obs)))
        print('Before add to buffer action : {}'.format(action))
        print('Before add to buffer reward : {}'.format(reward))
        print('Before add to buffer done : {}'.format(done_bool))
        replay_buffer.add((obs, new_obs, action, reward, done_bool))

          # We update the state, the episode timestep, the total timesteps, and the timesteps since the evaluation of the policy
        obs = new_obs
        episode_timesteps += 1
        total_timesteps += 1
        timesteps_since_eval += 1
        last_distance = distance
        last_reward = 0
        reward = 0

        if episode_timesteps % max_episode_steps == 0 :
          done = True


            

        
# Adding the painting tools

class MyPaintWidget(Widget):

    def on_touch_down(self, touch):
        global length, n_points, last_x, last_y
        with self.canvas:
            Color(0.8,0.7,0)
            d = 10.
            touch.ud['line'] = Line(points = (touch.x, touch.y), width = 10)
            last_x = int(touch.x)
            last_y = int(touch.y)
            n_points = 0
            length = 0
            sand[int(touch.x),int(touch.y)] = 1
            img = PILImage.fromarray(sand.astype("uint8")*255)
            img.save("./images/sand.jpg")

    def on_touch_move(self, touch):
        global length, n_points, last_x, last_y
        if touch.button == 'left':
            touch.ud['line'].points += [touch.x, touch.y]
            x = int(touch.x)
            y = int(touch.y)
            length += np.sqrt(max((x - last_x)**2 + (y - last_y)**2, 2))
            n_points += 1.
            density = n_points/(length)
            touch.ud['line'].width = int(20 * density + 1)
            sand[int(touch.x) - 10 : int(touch.x) + 10, int(touch.y) - 10 : int(touch.y) + 10] = 1

            
            last_x = x
            last_y = y

# Adding the API Buttons (clear, save and load)

class CarApp(App):

    def build(self):
        parent = Game()
        parent.serve_car()
        Clock.schedule_interval(parent.update, 1.0/60.0)
        #Clock.schedule_once(parent.update)
        #parent.update()
        self.painter = MyPaintWidget()
        clearbtn = Button(text = 'clear')
        savebtn = Button(text = 'save', pos = (parent.width, 0))
        loadbtn = Button(text = 'load', pos = (2 * parent.width, 0))
        clearbtn.bind(on_release = self.clear_canvas)
        savebtn.bind(on_release = self.save)
        loadbtn.bind(on_release = self.load)
        parent.add_widget(self.painter)
        parent.add_widget(clearbtn)
        parent.add_widget(savebtn)
        parent.add_widget(loadbtn)
        return parent

    def clear_canvas(self, obj):
        global sand
        self.painter.canvas.clear()
        sand = np.zeros((longueur,largeur))

    def save(self, obj):
        print("saving brain...")
        brain.save()
        plt.plot(scores)
        plt.show()

    def load(self, obj):
        print("loading last saved brain...")
        brain.load()

# Running the whole thing
if __name__ == '__main__':
    CarApp().run()
